### Objetivo:
Este projeto visa fornecer uma visão abrangente e detalhada das operações de comércio exterior do Brasil durante o ano de 2024, analisando tanto as importações quanto as exportações.

### Perguntas que o projeto busca responder:

1. Qual foi o valor total de importações (em CIF US$) e volume total de exportações (em peso) do Brasil em 2024?

2. Qual foi o valor total de importações (em CIF US$) e volume total de exportações (em peso) do Brasil em cada mês de 2024?

3. Quais foram os 10 principais países de origem das importações brasileiras em termos de valor CIF? Quais foram os 10 principais países de destino das exportações brasileiras em termos de peso (quilogramas)?

4. Quais foram os principais produtos importados e exportados pelo Brasil em 2024, identificados por código NCM e sua descrição?

5. Quais foram as vias de transporte mais utilizadas para as importações (considerando o valor CIF) e para as exportações (considerando o peso) em 2024?

6. Quais produtos foram exclusivamente importados pelo Brasil em 2024 (não houve exportação desses mesmos códigos NCM)?

7. Quais produtos foram exclusivamente exportados pelo Brasil em 2024 (não houve importação desses mesmos códigos NCM)?

8. Quais foram os 10 países com a maior balança comercial negativa e os 10 países com a maior balança comercial positiva em 2024?

9. Quais os produtos importados pelo Brasil do país que tem a balança comercial mais negativa em 2024?

In [0]:
%sql create database bronze;
--Criar database bronze

In [0]:
#Importar dados

from pyspark.sql.types import StructType, StructField, StringType

# Lista de informações dos arquivos CSV
arquivos = [
    {"location": "/FileStore/tables/CO_NCM.csv", "name": "CO_NCM"},
    {"location": "/FileStore/tables/CO_PAIS.csv", "name": "CO_PAIS"},
    {"location": "/FileStore/tables/CO_VIA.csv", "name": "CO_VIA"},
    {"location": "/FileStore/tables/CO_URF.csv", "name": "CO_URF"},
    {"location": "/FileStore/tables/CO_UNID.csv", "name": "CO_UNID"},
    {"location": "/FileStore/tables/IMP_2024.csv", "name": "IMP_2024"},
    {"location": "/FileStore/tables/EXP_2024.csv", "name": "EXP_2024"}
]

# Loop para processar cada arquivo
for arquivo in arquivos:
    # Configurações do arquivo CSV
    file_location = arquivo["location"]
    file_type = "csv"
    infer_schema = "true"
    first_row_is_header = "true"
    delimiter = ";"

    # Ler o arquivo CSV para obter os nomes das colunas e criar o schema
    df_temp = spark.read.format(file_type) \
        .option("header", first_row_is_header) \
        .option("sep", delimiter) \
        .option("inferSchema", "true") \
        .load(file_location)
    # Obter a lista de nomes de colunas
    colunas = df_temp.columns
    # Criar o esquema manualmente com todas as colunas como StringType
    schema = StructType([StructField(coluna, StringType(), True) for coluna in colunas])

    # Ler o arquivo CSV com o schema definido
    df = spark.read.format(file_type) \
        .option("header", first_row_is_header) \
        .option("sep", delimiter) \
        .schema(schema) \
        .load(file_location)

    # Criação da tabela gerenciada no schema bronze
    table_name = f"bronze.{arquivo['name']}"
    df.write.mode("overwrite").saveAsTable(table_name)

In [0]:
# Encerrar a sessão Spark
spark.stop

Out[3]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
#Verificar nulos

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("VerificarNulosMultiplasTabelas").getOrCreate()

tabelas = [
    "bronze.imp_2024",
    "bronze.co_ncm",
    "bronze.co_pais",
    "bronze.co_via",
    "bronze.co_urf",
    "bronze.co_unid",
    "bronze.exp_2024"
]

for tabela in tabelas:
    print(f"Verificando valores nulos na tabela: {tabela}")
    df = spark.table(tabela)
    for coluna in df.columns:
        existe_nulo = df.filter(col(coluna).isNull()).count() > 0
        if existe_nulo:
            print(f"  A coluna '{coluna}' contém valores nulos.")
        else:
            print(f"  A coluna '{coluna}' não contém valores nulos.")
    print("-" * 30)

Verificando valores nulos na tabela: bronze.imp_2024
  A coluna 'CO_ANO' não contém valores nulos.
  A coluna 'CO_MES' não contém valores nulos.
  A coluna 'CO_NCM' não contém valores nulos.
  A coluna 'CO_UNID' não contém valores nulos.
  A coluna 'CO_PAIS' não contém valores nulos.
  A coluna 'SG_UF_NCM' não contém valores nulos.
  A coluna 'CO_VIA' não contém valores nulos.
  A coluna 'CO_URF' não contém valores nulos.
  A coluna 'QT_ESTAT' não contém valores nulos.
  A coluna 'KG_LIQUIDO' não contém valores nulos.
  A coluna 'VL_FOB' não contém valores nulos.
  A coluna 'VL_FRETE' não contém valores nulos.
  A coluna 'VL_SEGURO' não contém valores nulos.
------------------------------
Verificando valores nulos na tabela: bronze.co_ncm
  A coluna 'CO_NCM' não contém valores nulos.
  A coluna 'NO_NCM_POR' contém valores nulos.
------------------------------
Verificando valores nulos na tabela: bronze.co_pais
  A coluna 'CO_PAIS' não contém valores nulos.
  A coluna 'CO_PAIS_ISOA3' nã

In [0]:
# Encerrar a sessão Spark
spark.stop

Out[5]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
%sql
--Analisar o valor nulo encontrado em co_ncm

select * from bronze.co_ncm
where NO_NCM_POR is null

CO_NCM,NO_NCM_POR
"Reagentes para determinação de glicose no sangue, sobre suporte em tiras, para uso direto""",null


In [0]:
#Verificar duplicadas

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

spark = SparkSession.builder.appName("VerificarDuplicatasMultiplasTabelas").getOrCreate()

tabelas = [
    "bronze.imp_2024",
    "bronze.co_ncm",
    "bronze.co_pais",
    "bronze.co_via",
    "bronze.co_urf",
    "bronze.co_unid",
    "bronze.exp_2024"
]

for tabela in tabelas:
    print(f"Verificando duplicatas na tabela: {tabela}")
    df = spark.table(tabela)
    total_linhas = df.count()
    linhas_distintas = df.distinct().count()
    duplicadas = total_linhas - linhas_distintas
    print(f"  Total de linhas: {total_linhas}")
    print(f"  Linhas distintas: {linhas_distintas}")
    print(f"  Linhas duplicadas: {duplicadas}")
    if duplicadas > 0:
        print("  Exemplo de duplicatas (limitado a 5):")
        df.groupBy(df.columns).agg(count("*").alias("count")).filter("count > 1").show(5)
    print("-" * 30)

Verificando duplicatas na tabela: bronze.imp_2024
  Total de linhas: 2273708
  Linhas distintas: 2273708
  Linhas duplicadas: 0
------------------------------
Verificando duplicatas na tabela: bronze.co_ncm
  Total de linhas: 13710
  Linhas distintas: 13710
  Linhas duplicadas: 0
------------------------------
Verificando duplicatas na tabela: bronze.co_pais
  Total de linhas: 281
  Linhas distintas: 281
  Linhas duplicadas: 0
------------------------------
Verificando duplicatas na tabela: bronze.co_via
  Total de linhas: 17
  Linhas distintas: 17
  Linhas duplicadas: 0
------------------------------
Verificando duplicatas na tabela: bronze.co_urf
  Total de linhas: 278
  Linhas distintas: 278
  Linhas duplicadas: 0
------------------------------
Verificando duplicatas na tabela: bronze.co_unid
  Total de linhas: 13720
  Linhas distintas: 13
  Linhas duplicadas: 13707
  Exemplo de duplicatas (limitado a 5):
+-------+--------------------+-------+-----+
|CO_UNID|             NO_UNID|SG_

In [0]:
# Encerrar a sessão Spark
spark.stop

Out[8]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
%sql
--Verificar siglas dos estados em imp_2024

SELECT * FROM bronze.imp_2024
WHERE SG_UF_NCM NOT IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
LIMIT 100

CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
2024,10,97019100,11,386,EX,04,0817600,1,25,27315,2633,0
2024,02,30049079,10,249,EX,04,0817700,1,1,10249,303,0
2024,10,87116000,11,160,EX,04,0717700,1,41,259,1039,0
2024,06,70139900,10,069,EX,04,0817700,111,111,4929,1448,0
2024,10,94038900,11,275,EX,04,0817700,200,200,16534,5186,0
2024,02,30049079,10,249,EX,04,0817600,0,0,73220,205,200
2024,04,85044090,11,249,EX,04,0817700,8,7,222,0,0
2024,09,30049079,10,399,EX,04,0817600,1,1,33900,79,0
2024,01,84186999,11,249,EX,04,0817600,1,40,2899,1030,0
2024,04,84021900,11,160,EX,01,0817800,1,450,3200,100,0


In [0]:
%sql
--Verificar siglas dos estados em exp_2024

SELECT * FROM bronze.exp_2024
WHERE SG_UF_NCM NOT IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
LIMIT 100

CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
2024,08,61044900,11,053,ND,04,0817600,1,1,859
2024,10,84198190,11,249,ND,04,0717700,6,194,40757
2024,11,44152000,11,169,ND,04,0227700,1,24,2
2024,10,42022100,11,573,ND,04,0817700,6,4,7833
2024,09,61099000,11,767,ND,04,0817700,40,14,2836
2024,06,85044010,11,097,ND,04,0817600,1,1,180
2024,01,62014000,11,232,ND,04,0817600,1,1,82
2024,05,62044200,11,573,ND,04,0817700,13,13,17148
2024,12,34012010,10,249,ND,04,0817700,7,7,1364
2024,04,61124900,11,023,ND,04,0817700,180,20,5823


In [0]:
%sql create database silver;
--Criar database silver

In [0]:
#Adicionar as tabelas ao schema silver

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CopiarTabelasParaSilver").getOrCreate()

tabelas_bronze = [
    "bronze.imp_2024",
    "bronze.exp_2024",
    "bronze.co_pais",
    "bronze.co_via",
    "bronze.co_urf"
]

for tabela_bronze in tabelas_bronze:
    tabela_silver = tabela_bronze.replace("bronze", "silver")
    df = spark.table(tabela_bronze)
    df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela_silver)

In [0]:
# Encerrar a sessão Spark
spark.stop

Out[13]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
#Ajustar tipo das colunas em imp_2024

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName("ConverterColunas").getOrCreate()
tabela_origem = "silver.imp_2024"
colunas_para_converter = ["VL_FRETE", "VL_FOB", "VL_SEGURO", "KG_LIQUIDO", "QT_ESTAT"]

df = spark.read.table(tabela_origem)

for coluna in colunas_para_converter:
    df = df.withColumn(coluna, col(coluna).cast(IntegerType()))

df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tabela_origem)

In [0]:
#Ajustar tipo das colunas em exp_2024

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName("ConverterColunas").getOrCreate()
tabela_origem = "silver.exp_2024"
colunas_para_converter = ["VL_FOB", "KG_LIQUIDO", "QT_ESTAT"]

df = spark.read.table(tabela_origem)

for coluna in colunas_para_converter:
    df = df.withColumn(coluna, col(coluna).cast(IntegerType()))

df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tabela_origem)

In [0]:
# Encerrar a sessão Spark
spark.stop

Out[16]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
#Verificar se os valores da base imp_2024 são todos positivos

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min

spark = SparkSession.builder.appName("VerificarPositivosImportacao").getOrCreate()
table_name = "silver.imp_2024"
columns_to_check = ["QT_ESTAT", "KG_LIQUIDO", "VL_FOB", "VL_FRETE", "VL_SEGURO"]
df = spark.table(table_name)

# Verifique se o valor mínimo em cada coluna é >= 0
min_values = df.select([min(col(c)).alias(c) for c in columns_to_check]).first().asDict()

all_positive = all(value >= 0 for value in min_values.values())

if all_positive:
    print(f"Todos os valores nas colunas {columns_to_check} são positivos.")
else:
    negative_columns = [col_name for col_name, min_val in min_values.items() if min_val < 0]
    print(f"Valores negativos encontrados nas colunas: {negative_columns}")

Todos os valores nas colunas ['QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE', 'VL_SEGURO'] são positivos.


In [0]:
#Verificar se os valores da base exp_2024 são todos positivos

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min

spark = SparkSession.builder.appName("VerificarPositivosExportacao").getOrCreate()
table_name = "silver.exp_2024"
columns_to_check = ["QT_ESTAT", "KG_LIQUIDO", "VL_FOB"]
df = spark.table(table_name)

# Verifique se o valor mínimo em cada coluna é >= 0
min_values = df.select([min(col(c)).alias(c) for c in columns_to_check]).first().asDict()

all_positive = all(value >= 0 for value in min_values.values())

if all_positive:
    print(f"Todos os valores nas colunas {columns_to_check} são positivos.")
else:
    negative_columns = [col_name for col_name, min_val in min_values.items() if min_val < 0]
    print(f"Valores negativos encontrados nas colunas: {negative_columns}")

Todos os valores nas colunas ['QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB'] são positivos.


In [0]:
# Encerrar a sessão Spark
spark.stop

Out[19]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
#Remover duplicadas da base co_unid

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RemoverDuplicatasColunas").getOrCreate()

tabela_origem = "bronze.co_unid"
tabela_destino = "silver.co_unid" # Ou a mesma tabela se quiser sobrescrever

df = spark.read.table(tabela_origem)

colunas_para_considerar = ["CO_UNID", "NO_UNID", "SG_UNID"]
df_sem_duplicatas = df.dropDuplicates(colunas_para_considerar)

df_sem_duplicatas.write \
    .mode("overwrite") \
    .saveAsTable(tabela_destino)

In [0]:
# Encerrar a sessão Spark
spark.stop

Out[21]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
#Remover valores nulos de co_ncm

from pyspark.sql import SparkSession

# Inicialize a SparkSession (se ainda não estiver ativa)
spark = SparkSession.builder.appName("ProcessarCO_NCM").getOrCreate()

# Carregue a tabela bronze.co_ncm como um DataFrame
bronze_co_ncm_df = spark.table("bronze.co_ncm")

# Remova as linhas onde a coluna "co_ncm_por" é nula
silver_co_ncm_df = bronze_co_ncm_df.filter(bronze_co_ncm_df["no_ncm_por"].isNotNull())

# Salve o DataFrame resultante na tabela silver.co_ncm, sobrescrevendo se já existir
silver_co_ncm_df.write.mode("overwrite").saveAsTable("silver.co_ncm")

In [0]:
#Criar coluna data

from pyspark.sql import SparkSession
from pyspark.sql.functions import make_date, lit

spark = SparkSession.builder.appName("UnirAnoMesParaData").getOrCreate()

tabelas_silver = ["silver.imp_2024", "silver.exp_2024"]

for tabela_silver in tabelas_silver:
    print(f"Processando tabela: {tabela_silver}")
    df = spark.table(tabela_silver)
    
    df_com_data = df.withColumn("DATA", make_date(df["CO_ANO"], df["CO_MES"], lit(1)))

    # Excluindo as colunas CO_ANO e CO_MES
    df_sem_ano_mes = df_com_data.drop("CO_ANO", "CO_MES","DT_IMPORTACAO","DATA_2")

    # Escrevendo o DataFrame de volta na tabela
    df_sem_ano_mes.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela_silver)

Processando tabela: silver.imp_2024
Processando tabela: silver.exp_2024


In [0]:
# Encerrar a sessão Spark
spark.stop

Out[24]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
%sql create database gold;
--Criar database gold

In [0]:
#Criar tabela flat de importacao

join_sql = """
SELECT
    imp.DATA,
    imp.CO_NCM,
    ncm.NO_NCM_POR,
    imp.CO_PAIS,
    pais.CO_PAIS_ISOA3,
    pais.NO_PAIS,
    imp.SG_UF_NCM,
    imp.CO_UNID,
    unid.NO_UNID,
    unid.SG_UNID,
    imp.CO_URF,
    urf.NO_URF,
    imp.CO_VIA,
    via.NO_VIA,
    imp.QT_ESTAT,
    imp.KG_LIQUIDO,
    imp.VL_FOB,
    imp.VL_FRETE,
    imp.VL_SEGURO,
    imp.VL_FOB + imp.VL_FRETE + imp.VL_SEGURO AS VL_CIF
FROM
    silver.imp_2024 imp
LEFT JOIN
    silver.co_ncm ncm ON imp.CO_NCM = ncm.CO_NCM
LEFT JOIN
    silver.co_pais pais ON imp.CO_PAIS = pais.CO_PAIS
LEFT JOIN
    silver.co_via via ON imp.CO_VIA = via.CO_VIA
LEFT JOIN
    silver.co_urf urf ON imp.CO_URF = urf.CO_URF
LEFT JOIN
    silver.co_unid unid ON imp.CO_UNID = unid.CO_UNID;
"""

joined_df = spark.sql(join_sql)

# Selecione explicitamente as colunas na ordem desejada para um novo DataFrame
selected_df = joined_df.select(
    "DATA",
    "CO_NCM",
    "NO_NCM_POR",
    "CO_PAIS",
    "CO_PAIS_ISOA3",
    "NO_PAIS",
    "SG_UF_NCM",
    "CO_UNID",
    "NO_UNID",
    "SG_UNID",
    "CO_URF",
    "NO_URF",
    "CO_VIA",
    "NO_VIA",
    "QT_ESTAT",
    "KG_LIQUIDO",
    "VL_FOB",
    "VL_FRETE",
    "VL_SEGURO",
    "VL_CIF"
)

# Salve o DataFrame selecionado
selected_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold.BD_importacao")

In [0]:
# Encerrar a sessão Spark
spark.stop

Out[28]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78a7a80e39a0>>

In [0]:
#Criar tabela fla de exportação

join_sql = """
SELECT
    exp.DATA,
    exp.CO_NCM,
    ncm.NO_NCM_POR,
    exp.CO_PAIS,
    pais.CO_PAIS_ISOA3,
    pais.NO_PAIS,
    exp.SG_UF_NCM,
    exp.CO_URF,
    urf.NO_URF,
    exp.CO_VIA,
    via.NO_VIA,
    exp.QT_ESTAT,
    exp.KG_LIQUIDO,
    exp.VL_FOB
FROM
    silver.exp_2024 exp
LEFT JOIN
    silver.co_ncm ncm ON exp.CO_NCM = ncm.CO_NCM
LEFT JOIN
    silver.co_pais pais ON exp.CO_PAIS = pais.CO_PAIS
LEFT JOIN
    silver.co_via via ON exp.CO_VIA = via.CO_VIA
LEFT JOIN
    silver.co_urf urf ON exp.CO_URF = urf.CO_URF
LEFT JOIN
    silver.co_unid unid ON exp.CO_UNID = unid.CO_UNID;
"""

joined_df = spark.sql(join_sql)

# Selecione explicitamente as colunas na ordem desejada para um novo DataFrame
selected_df = joined_df.select(
    "DATA",
    "CO_NCM",
    "NO_NCM_POR",
    "CO_PAIS",
    "CO_PAIS_ISOA3",
    "NO_PAIS",
    "SG_UF_NCM",
    "CO_URF",
    "NO_URF",
    "CO_VIA",
    "NO_VIA",
    "QT_ESTAT",
    "KG_LIQUIDO",
    "VL_FOB"
)

# Salve o DataFrame selecionado
selected_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold.BD_exportacao")

In [0]:
#Verificar se tem nulos nas tabelas flat

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("VerificarNulosMultiplasTabelas").getOrCreate()

tabelas = [
    "gold.bd_importacao",
    "gold.bd_exportacao"
]

for tabela in tabelas:
    print(f"Verificando valores nulos na tabela: {tabela}")
    df = spark.table(tabela)
    for coluna in df.columns:
        existe_nulo = df.filter(col(coluna).isNull()).count() > 0
        if existe_nulo:
            print(f"  A coluna '{coluna}' contém valores nulos.")
        else:
            print(f"  A coluna '{coluna}' não contém valores nulos.")
    print("-" * 30)

Verificando valores nulos na tabela: gold.bd_importacao
  A coluna 'DATA' não contém valores nulos.
  A coluna 'CO_NCM' não contém valores nulos.
  A coluna 'NO_NCM_POR' contém valores nulos.
  A coluna 'CO_PAIS' não contém valores nulos.
  A coluna 'CO_PAIS_ISOA3' não contém valores nulos.
  A coluna 'NO_PAIS' não contém valores nulos.
  A coluna 'SG_UF_NCM' não contém valores nulos.
  A coluna 'CO_UNID' não contém valores nulos.
  A coluna 'NO_UNID' não contém valores nulos.
  A coluna 'SG_UNID' não contém valores nulos.
  A coluna 'CO_URF' não contém valores nulos.
  A coluna 'NO_URF' não contém valores nulos.
  A coluna 'CO_VIA' não contém valores nulos.
  A coluna 'NO_VIA' não contém valores nulos.
  A coluna 'QT_ESTAT' não contém valores nulos.
  A coluna 'KG_LIQUIDO' não contém valores nulos.
  A coluna 'VL_FOB' não contém valores nulos.
  A coluna 'VL_FRETE' não contém valores nulos.
  A coluna 'VL_SEGURO' não contém valores nulos.
  A coluna 'VL_CIF' não contém valores nulos.


In [0]:
%sql
--Valor total de importações

SELECT
    SUM(VL_CIF) AS valor_total_importado,
    SUM(KG_LIQUIDO) AS kg_total_importado
FROM gold.BD_importacao
GROUP BY ALL;

valor_total_importado,kg_total_importado
277954083297,185124462140


In [0]:
%sql
--Valor total de exportações

SELECT
    SUM(KG_LIQUIDO) AS kg_total_exportado,
    SUM(VL_FOB) AS valor_total_exportado
FROM gold.BD_exportacao
GROUP BY ALL;

kg_total_exportado,valor_total_exportado
558962051049,337046161710


In [0]:
%sql
--Valor total de importações por mês

SELECT
    DATA,
    SUM(VL_CIF) AS valor_total_importado,
    SUM(KG_LIQUIDO) AS kg_total_importado
FROM gold.BD_importacao
GROUP BY ALL
ORDER BY DATA;

DATA,valor_total_importado,kg_total_importado
2024-01-01,21547872126,14271017741
2024-02-01,19114733823,11934058507
2024-03-01,21495520722,14131530601
2024-04-01,22979305733,14996217579
2024-05-01,22980758432,15314763004
2024-06-01,23614174503,15987102494
2024-07-01,24631610572,16852324457
2024-08-01,25743997254,17627190679
2024-09-01,24918269544,17357163569
2024-10-01,26863041835,18626111446


In [0]:
%sql
--Valor total de exportações por mês

SELECT
    DATA,
    SUM(KG_LIQUIDO) AS kg_total_exportado,
    SUM(VL_FOB) AS valor_total_exportado
FROM gold.BD_exportacao
GROUP BY ALL
ORDER BY DATA;

DATA,kg_total_exportado,valor_total_exportado
2024-01-01,41008790865,26702655353
2024-02-01,38357975430,23348254025
2024-03-01,49999567069,27657419417
2024-04-01,51584087920,30327524963
2024-05-01,50667003654,30190264650
2024-06-01,47727339606,28731639979
2024-07-01,53383018078,30841394890
2024-08-01,47864657915,28736329972
2024-09-01,48368034189,28471402288
2024-10-01,47022265242,29300866189


In [0]:
%sql
--Principais países de origem das importações

SELECT
    NO_PAIS AS pais_origem,
    SUM(VL_CIF) AS valor_total_importado,
    SUM(KG_LIQUIDO) AS kg_total_importado,
    ROUND((SUM(VL_CIF) * 100.0 / (SELECT SUM(VL_CIF) FROM gold.BD_importacao)),1) AS pct_participacao_valor
FROM gold.BD_importacao
GROUP BY NO_PAIS
ORDER BY valor_total_importado DESC
LIMIT 10;

pais_origem,valor_total_importado,kg_total_importado,pct_participacao_valor
China,69250007040,26737510808,24.9
Estados Unidos,42483615914,32327724687,15.3
Alemanha,14190331014,1968668072,5.1
Argentina,14083454256,11044038950,5.1
Rússia,12208621073,23337603811,4.4
Índia,7267341117,2123427413,2.6
Itália,6595552965,844880150,2.4
França,6320237752,743615219,2.3
México,6009301991,728495280,2.2
Japão,5672922369,896919885,2.0


In [0]:
%sql
--Principais países de destino das exportações

SELECT
    NO_PAIS AS pais_destino,
    SUM(KG_LIQUIDO) AS kg_total_exportado,
    SUM(VL_FOB) AS valor_total_exportado,
    ROUND((SUM(VL_FOB) * 100.0 / (SELECT SUM(VL_FOB) FROM gold.BD_exportacao)),1) AS pct_participacao_valor
FROM gold.BD_exportacao
GROUP BY NO_PAIS
ORDER BY valor_total_exportado DESC
LIMIT 10;

pais_destino,kg_total_exportado,valor_total_exportado,pct_participacao_valor
China,163094576867,94372036594,28.0
Estados Unidos,40616900178,40368569157,12.0
Argentina,8014571916,13777621257,4.1
Países Baixos (Holanda),24161088670,11720075021,3.5
Espanha,19467668436,9969843825,3.0
Singapura,10891090180,7874539509,2.3
México,6889336666,7802031715,2.3
Chile,5658427404,6657592071,2.0
Canadá,8292477454,6317347891,1.9
Alemanha,5250656157,5846881069,1.7


In [0]:
%sql
--Principais produtos exportados

SELECT
    NO_NCM_POR AS produto,
    SUM(VL_CIF) AS valor_total_importado,
    SUM(KG_LIQUIDO) AS kg_total_importado
FROM gold.BD_importacao
GROUP BY NO_NCM_POR
ORDER BY valor_total_importado DESC
LIMIT 10;

produto,valor_total_importado,kg_total_importado
Gasóleo (óleo diesel),9083984018,12028127218
Óleos brutos de petróleo,8940585469,13944653402
Partes de turborreatores ou de turbopropulsores,4848710598,310107
Outros cloretos de potássio,4116814967,13651091870
Turborreatores de empuxo superior a 25 kN,3459336873,2351149
"Outros produtos imunológicos, apresentados em doses ou acondicionados para venda a retalho",3349927462,666234
"Outros veículos automóveis com motor diesel, para carga <= 5 toneladas",3315536509,249046422
"Hulha betuminosa, não aglomerada",3088453110,15587414736
"Ureia, mesmo em solução aquosa, com teor de nitrogênio (azoto) superior a 45 %, em peso, calculado sobre o produto anidro no estado seco",2945689628,8309735040
Células fotovoltaicas montadas em módulos ou em painéis,2875444624,1237972436


In [0]:
%sql
--Principais produtos exportados

SELECT
    NO_NCM_POR AS produto,
    SUM(KG_LIQUIDO) AS kg_total_exportado,
    SUM(VL_FOB) AS valor_total_exportado
FROM gold.BD_exportacao
GROUP BY NO_NCM_POR
ORDER BY valor_total_exportado DESC
LIMIT 10;

produto,kg_total_exportado,valor_total_exportado
Óleos brutos de petróleo,86656210875,44963878520
"Soja, mesmo triturada, exceto para semeadura",98807843757,42941894470
"Minérios de ferro e seus concentrados, exceto as piritas de ferro ustuladas (cinzas de piritas), não aglomerados",114219853221,26574698749
Outros açúcares de cana,33474593276,15925320657
"Café não torrado, não descafeinado, em grão",2767297450,11331933183
"Carnes desossadas de bovino, congeladas",2236854795,10049077475
"Pastas químicas de madeira, à soda ou ao sulfato, exceto pastas para dissolução, semibranqueadas ou branqueadas, de não coníferas",18322102486,9551859761
"Milho em grão, exceto para semeadura",39744475173,8046317455
Fuel oil,13430519227,7375146645
"Bagaços e outros resíduos sólidos, da extração do óleo de soja",17657150092,7233993219


In [0]:
%sql
--Análise das vias de transporte mais utilizadas para importação

SELECT
    NO_VIA AS via_transporte,
    SUM(VL_CIF) AS valor_total_importado,
    SUM(KG_LIQUIDO) AS kg_total_importado,
    ROUND(SUM(VL_CIF)/SUM(KG_LIQUIDO),1) AS CIF_POR_KG
FROM gold.BD_importacao
GROUP BY NO_VIA
ORDER BY valor_total_importado DESC;

via_transporte,valor_total_importado,kg_total_importado,CIF_POR_KG
MARITIMA,206819878937,171948678927,1.2
AEREA,55363334331,262281528,211.1
RODOVIARIA,12175036052,8447084665,1.4
CONDUTO/REDE DE TRANSMISSAO,2328052982,3957039448,0.6
MEIOS PROPRIOS,691488222,411231639,1.7
ENTRADA/SAIDA FICTA,460208018,52838064,8.7
COURIER,102141816,10649,9591.7
FERROVIARIA,12987736,42312000,0.3
FLUVIAL,708898,2984349,0.2
POSTAL,246305,871,282.8


In [0]:
%sql
--Análise das vias de transporte mais utilizadas para exportação

SELECT
    NO_VIA AS via_transporte,
    SUM(KG_LIQUIDO) AS kg_total_exportado,
    SUM(VL_FOB) AS valor_total_exportado
FROM gold.BD_exportacao
GROUP BY NO_VIA
ORDER BY valor_total_exportado DESC;

via_transporte,kg_total_exportado,valor_total_exportado
MARITIMA,543823868321,298148235096
RODOVIARIA,7359749979,19263694598
AEREA,2584156236,17136024654
MEIOS PROPRIOS,55452682,1121279412
VIA NAO DECLARADA,296780944,602126554
FLUVIAL,3139998731,280896828
LACUSTRE,1454838881,208057679
CONDUTO/REDE DE TRANSMISSAO,1611,85604439
FERROVIARIA,102011230,73788029
VICINAL FRONTEIRICO,144934825,64829981


In [0]:
%sql
--Produtos que são apenas importados

SELECT DISTINCT
    imp.CO_NCM,
    imp.NO_NCM_POR,
    sum(imp.VL_CIF) AS SUM_CIF,
    sum(imp.KG_LIQUIDO) AS SUM_KG
FROM
    gold.BD_importacao imp
WHERE
    imp.CO_NCM NOT IN (SELECT DISTINCT exp.CO_NCM FROM gold.BD_exportacao exp)
GROUP BY ALL
ORDER BY SUM_CIF DESC;

CO_NCM,NO_NCM_POR,SUM_CIF,SUM_KG
87036000,"Outros veículos, equipados para propulsão, simultaneamente, com um motor de pistão alternativo de ignição por centelha (faísca) e um motor elétrico, suscetíveis de serem carregados por conexão a uma fonte externa de energia elétrica",2130175312,140695851
26131010,Molibdenita ustulada (minérios de molibdênio),355181205,12860000
88023021,"Aviões e outros veículos aéreos, a turboélice, multimotores, de peso inferior ou igual a 7.000 kg, vazios",280609059,314817
30021239,"Outras frações do sangue, preparadas como medicamentos",218661970,182862
88021210,"Helicópteros, de peso inferior ou igual a 3.500 kg",166675870,59771
87041010,"Dumpers para transporte de mercadoria >= 85 toneladas, utilizado fora de rodovias",151526083,7986759
88022021,"Aviões e outros veículos aéreos, a turboélice, de peso não superior a 2.000 kg, vazios, monomotores",134462092,112205
29309034,Ácido 2-hidroxi-4-(metiltio)butanóico e seu sal cálcico,130392821,81820335
87035000,"Outros veículos, equipados para propulsão, simultaneamente, com um motor de pistão de ignição por compressão (diesel ou semidiesel) e um motor elétrico, exceto os suscetíveis de serem carregados por conexão a uma fonte externa de energia elétrica",127678178,6066070
29173600,Ácido tereftálico e seus sais,127650977,138829437


In [0]:
%sql
--Produtos que são apenas exportados

SELECT DISTINCT
    exp.CO_NCM,
    exp.NO_NCM_POR,
    sum(exp.KG_LIQUIDO) AS SUM_KG
FROM
    gold.BD_exportacao exp
WHERE
    exp.CO_NCM NOT IN (SELECT DISTINCT imp.CO_NCM FROM gold.BD_importacao imp)
GROUP BY ALL
ORDER BY SUM_KG DESC;

CO_NCM,NO_NCM_POR,SUM_KG
26011210,"Minérios de ferro e seus concentrados, exceto as piritas de ferro ustuladas (cinzas de piritas), aglomerados por processo de peletização, de diâmetro superior ou igual a 8mm e inferior ou igual a 18mm",26124836972
23040090,"Bagaços e outros resíduos sólidos, da extração do óleo de soja",17657150092
26060011,Bauxita não calcinada (minério de alumínio),5240519373
20091200,"Suco (sumo) de laranja, não fermentados, sem adição de álcool, com ou sem adição de açúcar ou de outros edulcorantes, não congelado, com valor Brix não superior a 20",1884552692
02071220,"Carnes de galos/galinhas, não cortadas em pedaços, congeladas, sem miudezas",1033173866
44039800,"Madeira em bruto, mesmo descascada, desalburnada ou esquadriada, de eucalipto (Eucalyptus spp.)",937781524
01022990,Outros bovinos vivos,348416682
02071422,"Peitos desossados de galinha, comestíveis, congelados",330504204
20091100,"Suco (sumo) de laranja, não fermentados, sem adição de álcool, com ou sem adição de açúcar ou de outros edulcorantes, congelado",300248778
44034200,"Madeira em bruto, mesmo descascada, desalburnada ou esquadriada, de teca",269713871


In [0]:
%sql
--Top 10 países com a balança comercial positiva

SELECT
    COALESCE(imp_mensal.NO_PAIS, exp_mensal.NO_PAIS) AS pais,
    COALESCE(exp_mensal.total_exportado, 0) - COALESCE(imp_mensal.total_importado, 0) AS balanca_comercial
FROM
    (SELECT NO_PAIS, SUM(VL_CIF) AS total_importado FROM gold.BD_importacao GROUP BY 1) AS imp_mensal
FULL OUTER JOIN
    (SELECT NO_PAIS, SUM(VL_FOB) AS total_exportado FROM gold.BD_exportacao GROUP BY 1) AS exp_mensal
ON imp_mensal.NO_PAIS = exp_mensal.NO_PAIS
ORDER BY balanca_comercial DESC
LIMIT 10;

pais,balanca_comercial
China,25122029554
Países Baixos (Holanda),9321488371
Singapura,7034383689
Espanha,5800339755
Emirados Árabes Unidos,3607384124
Canadá,3292555651
Irã,2991704759
Egito,2958357679
Malásia,2646641201
Turquia,2461469149


In [0]:
%sql
--Top 10 países com a balança comercial negativa

SELECT
    COALESCE(imp_mensal.NO_PAIS, exp_mensal.NO_PAIS) AS pais,
    COALESCE(exp_mensal.total_exportado, 0) - COALESCE(imp_mensal.total_importado, 0) AS balanca_comercial
FROM
    (SELECT NO_PAIS, SUM(VL_CIF) AS total_importado FROM gold.BD_importacao GROUP BY 1) AS imp_mensal
FULL OUTER JOIN
    (SELECT NO_PAIS, SUM(VL_FOB) AS total_exportado FROM gold.BD_exportacao GROUP BY 1) AS exp_mensal
ON imp_mensal.NO_PAIS = exp_mensal.NO_PAIS
ORDER BY balanca_comercial ASC
LIMIT 10;

pais,balanca_comercial
Rússia,-10758580216
Alemanha,-8343449945
França,-3356428378
Itália,-2128737721
Estados Unidos,-2115046757
Índia,-1994021013
Suíça,-1824293109
Suécia,-1566769512
Áustria,-1370061630
Dinamarca,-1174694140


In [0]:
%sql
--Produtos importados do país com maior deficit

SELECT
    NO_NCM_POR AS produto,
    SUM(VL_CIF) AS valor_total_importado,
    SUM(KG_LIQUIDO) AS kg_total_importado
FROM gold.BD_importacao
WHERE NO_PAIS = "Rússia"
GROUP BY NO_NCM_POR
ORDER BY valor_total_importado DESC
LIMIT 10;

produto,valor_total_importado,kg_total_importado
Gasóleo (óleo diesel),5968345995,7871058421
Outros cloretos de potássio,1597389991,5411140069
"Diidrogeno-ortofosfato de amônio (fosfato monoamônico ou monoamoniacal), mesmo misturado com hidrogeno-ortofosfato de diamônio (fosfato diamônico ou diamoniacal)",1338096754,2233706359
"Ureia, mesmo em solução aquosa, com teor de nitrogênio (azoto) superior a 45 %, em peso, calculado sobre o produto anidro no estado seco",550087825,1506625105
"Outras naftas, exceto para petroquímica",489344805,634824391
"Outras gasolinas, exceto para aviação",435523910,587096620
"Outros produtos semimanufaturados de ferro ou aço não ligado, de seção transversal retangular, que contenham, em peso, menos de 0,25 % de carbono",364253648,664943380
"Adubos (fertilizantes) minerais ou químicos, que contenham os três elementos fertilizantes: nitrogênio (azoto), fósforo e potássio",315201699,908380175
"Nitrato de amônio, mesmo em solução aquosa",272432306,1076513675
"Outros trigos e misturas de trigo com centeio, exceto para semeadura",188087450,711577536
